# Mini Tensorflow Framework in Python

In [1]:
class Operation():
    """
    A Operation is a node in a Graph. Tensorflow uses the same concept of a Graph.
    
    Operation class will be inherited by the other classes that actually compute the sepecific operation,
    like addition and multiplication
    """
    def __init__(self, input_nodes = []):
        """
            Initalize the Operation 
        """
        self.input_nodes = input_nodes # the list of input nodes
        self.output_nodes = [] # list of nodes to consuming the node's output
        
        # For every node in the input, we append this operation (self) to the list of
        # the consumers of the input nodes
        
        # For every input node in the input, append this operation (self) to the list of the consumers of the input node
        for node in input_nodes:
            node.output_nodes.append(self)
            
        # There will be a global default graph (TensorFlow works this way)
        # We will then append this particular operation
        # Append this operation to the list of operations in the currently active default graph
        _default_graph.operations.append(self)
            
    def compute(self):
        '''
        This is a placeholder function. It will be overwritten by the actual specific operation
        that inherits from this class.
        '''
        pass
    
    

In [2]:
#Additon Class
class add(Operation):
    
    def __init__(self, x, y):
        
        super().__init__([x, y]) # initialize the operation class constructor
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var+y_var

In [3]:
#Multiplication Class
class multiply(Operation):
    
    def __init__(self, a, b):
        
        super().__init__([a,b]) # initialize the operation class constructor
        
    def compute(self, a_var, b_var):
        
        self.inputs = [a_var, b_var]
        return a_var * b_var

In [4]:
#matrix multiplication class
class matmul(Operation):
    
    def __init__(self, a_mat, b_mat):
        
        super().__init__([a_mat, b_mat]) # initialize the operation class constructor
        
    def compute(self, a_mat, b_mat):
        
        self.inputs = [a_mat, b_mat]
        return a_mat.dot(b_mat)

In [5]:
class Placeholder():
    """
    A Placeholder is a node that needs to be provided a value for computing the object in the Graph
    """
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [6]:
class Variable():
    """
    A variable is the changeable parameter of the Graph
    """
    
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

In [7]:
class Graph():
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        """
        Sets this Graph Instance as the Global Default Graph
        """
        global _default_graph
        _default_graph = self
        

## A Basic Graph

$$ z = Ax + b $$

With A = 10 and b = 1

$$ z = 10x + 1 $$

Here we just need a Placeholder for x and then we can solve it.

In [8]:
# Lets get started
g = Graph() # create an object g from Graph class
g.set_as_default() # set the graph instance as globally 

In [9]:
# Create the Variable A and b
A = Variable(10)
b = Variable(1)

In [10]:
#Will fill the value later 
x = Placeholder()

In [11]:
y = multiply(A,x)

In [12]:
z = add(y,b)

# Session 

In [13]:
import numpy as np

In [14]:
# Traversing Operation Nodes

def traverse_postorder(operation):
    """ 
    PostOrder Traversal of Nodes. This makes sure computations are done in 
    the correct order (Ax first , then Ax + b). 
    """
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [15]:
class Session():
    
    def run(self, operation, feed_dict={}):
        """
        Operation: to compute
        feed_dict : to pass the value to placeholder 
        """
        
        #put nodes in correct order
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            
            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
            
            elif type(node) == Variable:
                
                node.output = node.value
                
            else :
                
                node.inputs = [input_node.output for input_node in node.input_nodes]
                
                node.output = node.compute(*node.inputs)
                
            # Convert lists to numpy arrays
            if type(node.output) == list:
                node.output = np.array(node.output)
         
        #return the requested node
        return operation.output

In [16]:
sess = Session()

In [17]:
result = sess.run(operation=z,feed_dict={x:10})

In [18]:
print(result)

101


In [19]:
10*10 + 1

101

In [20]:
# Lets try another example of matrix multiplication
g = Graph() 

g.set_as_default() 

A = Variable([[10,10],[20,20]]) 
b = Variable([1,1])

x = Placeholder()

y = matmul(A, x)
z = add(y, b)

In [21]:
sess = Session()

result = sess.run(operation=z, feed_dict={x:10})
result

array([[101, 101],
       [201, 201]])

# Looks like we created our own mini tensorflow framework